In [22]:
import numpy as np
import math
import pandas as pd
import matplotlib.pyplot as plt
from scipy.special import logit
from scipy.stats import norm

import tensorflow as tf
from keras import layers, models, datasets

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
from torchinfo import summary
from torch.optim.lr_scheduler import StepLR
import torch.autograd.profiler as profiler

from sklearn.model_selection import train_test_split, LeaveOneOut, StratifiedKFold, cross_val_predict
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder, PowerTransformer
from sklearn.metrics import f1_score, log_loss, accuracy_score
from sklearn.linear_model import LogisticRegression

import sys
import time

from learnable_activation import LearnableActivation

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [18]:
pip install learnable-activation

Note: you may need to restart the kernel to use updated packages.


An interpolation based learning technique, driven through explicit regularization

In [3]:
def calculate_metrics(model, data_tensor, labels_tensor, batch_size=1024, num_features=22):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for start_idx in range(0, len(data_tensor), batch_size):
            end_idx = min(start_idx + batch_size, len(data_tensor))
            inputs = data_tensor[start_idx:end_idx].view(-1, num_features)
            labels = labels_tensor[start_idx:end_idx]

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds, average='weighted')
    return accuracy, f1

In [4]:
class CustomDataLoader:
    def __init__(self, features, labels, validation_size=0.2, random_state=42):
        if validation_size > 0.0:
            train_data, val_data, train_labels, val_labels = train_test_split(
                features, labels, test_size=validation_size, stratify=labels, random_state=random_state
            )
            
            self.val_data_tensor = torch.tensor(val_data).float().to(device)
            self.val_labels_tensor = torch.tensor(val_labels).long().to(device)
        else:
            train_data, train_labels = features, labels
            self.val_data_tensor, self.val_labels_tensor = None, None
        
        self.train_data_tensor = torch.tensor(train_data).float().to(device)
        self.train_labels_tensor = torch.tensor(train_labels).long().to(device)

        torch.manual_seed(random_state)

        indices = torch.randperm(len(self.train_data_tensor))

        self.train_data_tensor = self.train_data_tensor[indices]
        self.train_labels_tensor = self.train_labels_tensor[indices]

In [34]:
def evaluate_model(model, custom_train_loader, criterion, optimizer, num_epochs, scheduler, batch_size=1024, num_features=22):
    unregularized_criterion = nn.CrossEntropyLoss()

    for epoch in range(num_epochs):
        running_loss = 0.0
        
        model.train()
        for start_idx in range(0, len(custom_train_loader.train_data_tensor), batch_size):
            end_idx = min(start_idx + batch_size, len(custom_train_loader.train_data_tensor))
            inputs = custom_train_loader.train_data_tensor[start_idx:end_idx].view(-1, num_features)
            labels = custom_train_loader.train_labels_tensor[start_idx:end_idx]

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels, model)
            loss.backward()
            optimizer.step()
            scheduler.step()
            running_loss += loss.item() * len(labels)
                      
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for start_idx in range(0, len(custom_train_loader.val_data_tensor), batch_size):
                end_idx = min(start_idx + batch_size, len(custom_train_loader.val_data_tensor))
                val_inputs = custom_train_loader.val_data_tensor[start_idx:end_idx].view(-1, num_features)
                val_labels = custom_train_loader.val_labels_tensor[start_idx:end_idx]

                val_outputs = model(val_inputs)
                val_loss += unregularized_criterion(val_outputs, val_labels).item() * len(val_labels)

        avg_train_loss = running_loss / len(custom_train_loader.train_data_tensor)
#         print(f'Epoch {epoch + 1}: {avg_train_loss}')
        avg_val_loss = val_loss / len(custom_train_loader.val_data_tensor)

        train_accuracy, train_f1 = calculate_metrics(model, custom_train_loader.train_data_tensor, custom_train_loader.train_labels_tensor, batch_size, num_features)
        val_accuracy, val_f1 = calculate_metrics(model, custom_train_loader.val_data_tensor, custom_train_loader.val_labels_tensor, batch_size, num_features)

        print(f'Epoch {epoch + 1}, Training Loss: {avg_train_loss}, Validation Loss: {avg_val_loss}')
        print(f'Training Accuracy: {train_accuracy}, Training F1 Score: {train_f1}')
        print(f'Validation Accuracy: {val_accuracy}, Validation F1 Score: {val_f1}')
        print()


In [35]:
class CustomLoss(nn.Module):
    def __init__(self, criterion, l1_lambda=0.0, l2_lambda=0.0, f1_lambda=0.0, f2_lambda=0.0):
        super(CustomLoss, self).__init__()
        self.criterion = criterion
        self.l1_lambda = l1_lambda
        self.l2_lambda = l2_lambda
        self.f1_lambda = f1_lambda
        self.f2_lambda = f2_lambda

    def forward(self, outputs, labels, model):
        l1_norm = sum(
            p.abs().sum()
            for name, module in model.named_modules()
            if isinstance(module, nn.Linear)
            for p in module.parameters()
            if "bias" not in name
        )
        l1_loss = self.l1_lambda * l1_norm

        l2_norm = sum(
            p.pow(2.0).sum()
            for name, module in model.named_modules()
            if isinstance(module, nn.Linear)
            for p in module.parameters()
            if "bias" not in name
        )
        l2_loss = self.l2_lambda * l2_norm

        f1_loss = 0
        f2_loss = 0
        for name, module in model.named_modules():
            if isinstance(module, LearnableActivation):
                interp_tensor = module.interp_tensor

                f1_diff = interp_tensor[:, 1:] - interp_tensor[:, :-1]
                f1_loss += self.f1_lambda * f1_diff.abs().sum()

                f2_diff = f1_diff[:, 1:] - f1_diff[:, :-1]
                f2_loss += self.f2_lambda * f2_diff.abs().sum()

        return self.criterion(outputs, labels) + l1_loss + l2_loss + f1_loss + f2_loss

    def regular_loss(self, outputs, labels):
        return self.criterion(outputs, labels)

In [16]:
data_dl = pd.read_csv('/kaggle/input/playground-series-s4e10/train.csv')
data_og = pd.read_csv('/kaggle/input/loan-approval-prediction/credit_risk_dataset.csv')

data_dl = data_dl.drop(["id"], axis=1)
data_og['person_emp_length'] = data_og['person_emp_length'].fillna(data_og['person_emp_length'].mean())
data_og['loan_int_rate'] = data_og['loan_int_rate'].fillna(data_og['loan_int_rate'].mean())

data_dl['source'] = 0
data_og['source'] = 1

data = pd.concat([data_dl, data_og], ignore_index=True)

grade_mapping = {'A': 1, 'B': 2, 'C': 3, 'D': 4, 'E': 5, 'F': 6, 'G': 7}
data['loan_grade'] = data['loan_grade'].map(grade_mapping)

purpose_mapping = {
    'DEBTCONSOLIDATION': 1,
    'HOMEIMPROVEMENT': 2,
    'MEDICAL': 3,
    'PERSONAL': 4,
    'EDUCATION': 5,
    'VENTURE': 6
}
data['loan_intent'] = data['loan_intent'].map(purpose_mapping)

home_ownership_mapping = {
    'OWN': 1,
    'MORTGAGE': 2,
    'OTHER': 3,
    'RENT': 4
}
data['person_home_ownership'] = data['person_home_ownership'].map(home_ownership_mapping)

print(data.columns)
print(data.isnull().sum())

X = data.drop(["loan_status"], axis=1)
X['loan_percent_income_ratio'] = (X['person_income'] / X['loan_amnt'])
X['loan_to_income_ratio'] = X['loan_amnt'] / X['person_income']
X['financial_burden'] = X['loan_amnt'] * X['loan_int_rate']
X['loan_int_emp_interaction'] = X['loan_int_rate'] * X['person_emp_length']
X['debt_to_credit_ratio'] = X['loan_amnt'] / X['cb_person_cred_hist_length']

y = data["loan_status"]

exclude_columns = [
    "loan_percent_income_ratio",
    "loan_to_income_ratio",
    "financial_burden",
    "debt_to_credit_ratio"
]

X = pd.get_dummies(X, drop_first=True)

for col in X.columns:
    if col not in exclude_columns:
        if (X[col] > 0).all():
            X[col] = np.log(X[col])

print(X.shape, y.shape)
print(X.columns)

Index(['person_age', 'person_income', 'person_home_ownership',
       'person_emp_length', 'loan_intent', 'loan_grade', 'loan_amnt',
       'loan_int_rate', 'loan_percent_income', 'cb_person_default_on_file',
       'cb_person_cred_hist_length', 'loan_status', 'source'],
      dtype='object')
person_age                    0
person_income                 0
person_home_ownership         0
person_emp_length             0
loan_intent                   0
loan_grade                    0
loan_amnt                     0
loan_int_rate                 0
loan_percent_income           0
cb_person_default_on_file     0
cb_person_cred_hist_length    0
loan_status                   0
source                        0
dtype: int64
(91226, 17) (91226,)
Index(['person_age', 'person_income', 'person_home_ownership',
       'person_emp_length', 'loan_intent', 'loan_grade', 'loan_amnt',
       'loan_int_rate', 'loan_percent_income', 'cb_person_cred_hist_length',
       'source', 'loan_percent_income_ratio', 

In [8]:
x_scaler = StandardScaler()
x_scaled = x_scaler.fit_transform(X)

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

print(x_scaled.shape)

(91226, 17)


In [25]:
class CustomLoss(nn.Module):
    def __init__(self, base_loss_fn, first_order_weight=0.0, second_order_weight=0.0):
        super(CustomLoss, self).__init__()
        self.first_order_weight = first_order_weight
        self.second_order_weight = second_order_weight
        self.base_loss_fn = base_loss_fn

    def forward(self, outputs, labels, optimized_tensor):
        base_loss = self.base_loss_fn(outputs, labels)
        
        first_order_x = (optimized_tensor[:, :, :-1, :] - optimized_tensor[:, :, 1:, :])
        first_order_y = (optimized_tensor[:, :, :, :-1] - optimized_tensor[:, :, :, 1:])
        first_order_loss = self.first_order_weight * (torch.sum(first_order_x ** 2) + torch.sum(first_order_y ** 2))
        
        second_order_x = (first_order_x[:, :, :-1:, :] - first_order_x[:, :, 1:, :])
        second_order_y = (first_order_x[:, :, :, :-1] - first_order_x[:, :, :, 1:])
        second_order_loss = self.second_order_weight * (torch.sum(second_order_x ** 2) + torch.sum(second_order_y ** 2))

        return base_loss + first_order_loss + second_order_loss

In [10]:
import pandas as pd

# Assuming X is a DataFrame, retrieve the feature names
feature_names = X.columns

# Step 1: Scale the features
x_scaler = StandardScaler()
x_scaled = x_scaler.fit_transform(X)

# Step 2: Encode the labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Step 3: Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(x_scaled, y_encoded, test_size=0.2, random_state=42)

# Step 4: Initialize and train the Logistic Regression model
log_reg = LogisticRegression(penalty=None)
log_reg.fit(X_train, y_train)

# Step 5: Make predictions for accuracy and log loss
y_train_pred_proba = log_reg.predict_proba(X_train)
y_test_pred_proba = log_reg.predict_proba(X_test)
y_pred = log_reg.predict(X_test)

# Step 6: Evaluate the model
# Accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")

# Classification report
print("Classification Report:")
print(classification_report(y_test, y_pred))

# Log loss
train_log_loss = log_loss(y_train, y_train_pred_proba)
test_log_loss = log_loss(y_test, y_test_pred_proba)
print(f"Training Log Loss: {train_log_loss:.4f}")
print(f"Testing Log Loss: {test_log_loss:.4f}")

# Display feature coefficients
coefficients = log_reg.coef_[0]  # Assuming binary classification, adjust if multi-class
coef_df = pd.DataFrame({"Feature": feature_names, "Coefficient": coefficients})
print(coef_df)

Accuracy: 88.31%
Classification Report:


NameError: name 'classification_report' is not defined

In [53]:
features = concatenated_result.detach().cpu().numpy()


In [54]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Step 1: Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, y, test_size=0.2, random_state=0)

# Step 2: Initialize the Gradient Boosted Tree model
gbt_model = GradientBoostingClassifier(random_state=42)

# Step 3: Train the model
gbt_model.fit(X_train, y_train)

# Step 4: Make predictions
y_pred = gbt_model.predict(X_test)

# Step 5: Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)
    
print("Accuracy:", accuracy)
print("Confusion Matrix:\n", conf_matrix)
print("Classification Report:\n", class_report)

Accuracy: 0.9434396580072345
Confusion Matrix:
 [[15032   136]
 [  896  2182]]
Classification Report:
               precision    recall  f1-score   support

           0       0.94      0.99      0.97     15168
           1       0.94      0.71      0.81      3078

    accuracy                           0.94     18246
   macro avg       0.94      0.85      0.89     18246
weighted avg       0.94      0.94      0.94     18246



In [46]:
class TabularDenseNet(nn.Module):
    def __init__(self, input_size, output_size, num_layers=2, width=2, density=10):
        super(TabularDenseNet, self).__init__()

        self.layers = nn.ModuleList()
        self.activations = nn.ModuleList()

        for i in range(num_layers):
            self.activations.append(LearnableActivation(input_size, width, density))
            self.layers.append(nn.Linear(input_size, input_size, bias=False))

            with torch.no_grad():
                self.layers[-1].weight.copy_(torch.eye(input_size))

            input_size *= 2

        self.activation_second_last_layer = LearnableActivation(input_size, width, density)
        self.last_layer = nn.Linear(input_size, output_size, bias=False)

        with torch.no_grad():
            self.last_layer.weight.copy_(torch.zeros(output_size, input_size))

        self.activation_last_layer = LearnableActivation(output_size, width, density)
        
        print(LearnableActivation.__module__)

    def forward(self, x):
        outputs = [x]

        for i in range(len(self.layers)):
            concatenated_outputs = torch.cat(outputs, dim=1)
            outputs.append(self.layers[i](self.activations[i](concatenated_outputs)))

        outputs = torch.cat(outputs, dim=1)
        outputs = self.activation_second_last_layer(outputs)
        outputs = self.last_layer(outputs)
        outputs = self.activation_last_layer(outputs)
        return outputs.squeeze()
    
    def extract_features(self, x):
        # Forward pass up to the first layer's activation output
        concatenated_outputs = torch.cat([x], dim=1)  # Initial input
        activated_output = self.activations[0](concatenated_outputs)
        first_layer_output = self.layers[0](activated_output)
        
        return first_layer_output

In [47]:
class CustomLoss(nn.Module):
    def __init__(self, criterion, l1_lambda=0.0, l2_lambda=0.0, f1_lambda=0.0, f2_lambda=0.0):
        super(CustomLoss, self).__init__()
        self.criterion = criterion
        self.l1_lambda = l1_lambda
        self.l2_lambda = l2_lambda
        self.f1_lambda = f1_lambda
        self.f2_lambda = f2_lambda

    def forward(self, outputs, labels, model):
        l1_norm = sum(
            p.abs().sum()
            for name, module in model.named_modules()
            if isinstance(module, nn.Linear)
            for p in module.parameters()
            if "bias" not in name
        )
        l1_loss = self.l1_lambda * l1_norm

        l2_norm = sum(
            p.pow(2.0).sum()
            for name, module in model.named_modules()
            if isinstance(module, nn.Linear)
            for p in module.parameters()
            if "bias" not in name
        )
        l2_loss = self.l2_lambda * l2_norm

        f1_loss = 0
        f2_loss = 0
        for name, module in model.named_modules():
            if isinstance(module, LearnableActivation):
                interp_tensor = module.interp_tensor

                f1_diff = interp_tensor[:, 1:] - interp_tensor[:, :-1]
                f1_loss += self.f1_lambda * f1_diff.abs().sum()

                f2_diff = f1_diff[:, 1:] - f1_diff[:, :-1]
                f2_loss += self.f2_lambda * f2_diff.abs().sum()

        return self.criterion(outputs, labels) + l1_loss + l2_loss + f1_loss + f2_loss

    def regular_loss(self, outputs, labels):
        return self.criterion(outputs, labels)


In [48]:
num_epochs = 1
num_features = 17
num_classes = 2

model = TabularDenseNet(num_features, num_classes, 1).to(device)
criterion = CustomLoss(nn.CrossEntropyLoss(), 0.001 * 0, 0.001 * 0, 0.001 * 0, 0.001 * 0)
criterion = CustomLoss(nn.CrossEntropyLoss(), 0.001 * 1.0, 0.001 * 0.1, 0.001 * 0.01, 0.001 * 0.005)
criterion = CustomLoss(nn.CrossEntropyLoss(), 0.001 * 1.0, 0.001 * 0.1, 0.001 * 0.01, 0.001 * 0.005)
custom_train_loader = CustomDataLoader(x_scaled, y_encoded, validation_size=0.2, random_state=0)
criterion = CustomLoss(nn.CrossEntropyLoss(), 0.001 * 0, 0.001 * 0, 0.001 * 0, 0.001 * 0)

for name, param in model.named_parameters():
    break
    print(f"Layer: {name}")
    print(f"Shape: {param.shape}")
    print(param)
    
total_params = sum(p.numel() for p in model.parameters())
print(f'Total number of parameters: {total_params}')

# 3 layers 0.001 * 0.5 l1 val loss = 0.16691114300471785
# 2 layers no regularization Epoch 75, Training Loss: 0.15852869824943036, Validation Loss: 0.18101713914600992

learnable_activation.learnable_activation
Total number of parameters: 1470


In [49]:
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler

optimizer = optim.Adam(model.parameters(), lr=0.001 * 10)
scheduler = lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.9995)

start_time = time.time()

evaluate_model(model, custom_train_loader, criterion, optimizer, 150, scheduler, 1024 * 16, num_features)

elapsed_time = time.time() - start_time
print(f"Execution time: {elapsed_time:.6f} seconds")

Epoch 1, Training Loss: 0.6284685704570236, Validation Loss: 0.5040036569715238
Training Accuracy: 0.7461359276514113, Training F1 Score: 0.775094988179223
Validation Accuracy: 0.7471226570207169, Validation F1 Score: 0.7759626395948441

Epoch 2, Training Loss: 0.45116154482737736, Validation Loss: 0.3656772183480786
Training Accuracy: 0.8371197588380378, Training F1 Score: 0.8374865932775295
Validation Accuracy: 0.8397456976871643, Validation F1 Score: 0.8403969035158311

Epoch 3, Training Loss: 0.3537841805793645, Validation Loss: 0.32507977159115015
Training Accuracy: 0.8631542888462592, Training F1 Score: 0.8430357767051797
Validation Accuracy: 0.8627644415214294, Validation F1 Score: 0.8431513635340138

Epoch 4, Training Loss: 0.31331122279428525, Validation Loss: 0.288688952291298
Training Accuracy: 0.8865031515483695, Training F1 Score: 0.8798749419444711
Validation Accuracy: 0.88726296174504, Validation F1 Score: 0.8810340356310372

Epoch 5, Training Loss: 0.28311647448385535, 

In [51]:
x_scaled_tensor = torch.tensor(x_scaled, dtype=torch.float32)

# Now you can pass it to the model
outputs = model.extract_features(x_scaled_tensor)
print(outputs.shape)

torch.Size([91226, 17])


In [52]:
# Ensure x_scaled is a tensor
x_scaled_tensor = torch.tensor(x_scaled, dtype=torch.float32) if not torch.is_tensor(x_scaled) else x_scaled

# Concatenate along the feature dimension (dim=1), assuming both have the same batch size
concatenated_result = torch.cat((x_scaled_tensor, outputs), dim=1)
print(concatenated_result.shape)

torch.Size([91226, 34])


In [30]:
class TestClass(torch.nn.Module):
    def __init__(self, control_points, num_features, num_classes):
        super(TestClass, self).__init__()
        self.control_points = control_points
        self.num_features = num_features
        self.num_classes = num_classes
        self.num_pairs = num_features * (num_features - 1) // 2
        
        self.copy_tensor = nn.Parameter(torch.zeros(self.num_pairs, self.num_classes, self.control_points + 2, self.control_points + 2))
        self.i_indices, self.j_indices = torch.triu_indices(num_features, num_features, offset=1).to(device)
        self.feature_idx = torch.arange(self.num_pairs).view(1, -1, 1, 1).to(device)
        self.class_idx = torch.arange(self.num_classes).view(1, 1, -1, 1).to(device)
        
    def forward(self, x):
        x = torch.sigmoid(x)
        
        batch_size = x.shape[0]
        feature_idx = self.feature_idx.expand(batch_size, -1, self.num_classes, 1)
        class_idx = self.class_idx.expand(batch_size, self.num_pairs, -1, 1)
        x = x * self.control_points
        
        x_i = x[:, self.i_indices]
        x_j = x[:, self.j_indices]        
        feature_pairs = torch.stack([x_i, x_j], dim=2)
        
        lower_idx = torch.floor(feature_pairs).long()
        upper_idx = lower_idx + 1
        
        lower_idx_i_og = lower_idx[:, :, 0]
        upper_idx_i_og = upper_idx[:, :, 0]
        lower_idx_j_og = lower_idx[:, :, 1]
        upper_idx_j_og = upper_idx[:, :, 1]
        
        lower_idx_i = lower_idx_i_og.unsqueeze(-1).unsqueeze(-1)
        upper_idx_i = upper_idx_i_og.unsqueeze(-1).unsqueeze(-1)
        lower_idx_j = lower_idx_j_og.unsqueeze(-1).unsqueeze(-1)
        upper_idx_j = upper_idx_j_og.unsqueeze(-1).unsqueeze(-1)
        
        index_0_0 = self.copy_tensor[feature_idx, class_idx, lower_idx_i, lower_idx_j].squeeze(-1)
        index_0_1 = self.copy_tensor[feature_idx, class_idx, lower_idx_i, upper_idx_j].squeeze(-1)
        index_1_0 = self.copy_tensor[feature_idx, class_idx, upper_idx_i, lower_idx_j].squeeze(-1)
        index_1_1 = self.copy_tensor[feature_idx, class_idx, upper_idx_i, upper_idx_j].squeeze(-1)
        
        lower_weight_i = (x_i - lower_idx_i_og.float()).unsqueeze(-1)
        lower_weight_j = (x_j - lower_idx_j_og.float()).unsqueeze(-1)
        upper_weight_i = (x_i - upper_idx_i_og.float()).unsqueeze(-1)
        upper_weight_j = (x_j - upper_idx_j_og.float()).unsqueeze(-1)

        lower_triangle_interpolated = index_0_0 + (index_1_0 - index_0_0) * lower_weight_i + (index_0_1 - index_0_0) * lower_weight_j
        upper_triangle_interpolated = index_1_1 + (index_1_1 - index_0_1) * upper_weight_i + (index_1_1 - index_1_0) * upper_weight_j
        selected_triangle = torch.where(lower_weight_i + lower_weight_j < 1, lower_triangle_interpolated, upper_triangle_interpolated).sum(dim=-2)
        return selected_triangle

In [ ]:
num_epochs = 1
num_features = 17
num_classes = 2

model = TestClass(20, num_features, num_classes).to(device)
criterion = nn.CrossEntropyLoss()
custom_train_loader = CustomDataLoader(x_scaled, y_encoded, validation_size=0.2, random_state=0)

for name, param in model.named_parameters():
    break
    print(f"Layer: {name}")
    print(f"Shape: {param.shape}")
    print(param)
    
total_params = sum(p.numel() for p in model.parameters())
print(f'Total number of parameters: {total_params}')

# 3 layers 0.001 * 0.5 l1 val loss = 0.16691114300471785
# 2 layers no regularization Epoch 75, Training Loss: 0.15852869824943036, Validation Loss: 0.18101713914600992

In [ ]:
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler

optimizer = optim.Adam(model.parameters(), lr=0.001 * 10 * 2)
scheduler = lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.9995)

torch.cuda.synchronize()
start_time = time.time()

evaluate_model(model, custom_train_loader, criterion, optimizer, 150, scheduler, 1024 * 16, num_features)

elapsed_time = time.time() - start_time
print(f"Execution time: {elapsed_time:.6f} seconds")

In [ ]:
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler

optimizer = optim.Adam(model.parameters(), lr=0.001 * 10 * 2)
scheduler = lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.9995)

torch.cuda.synchronize()
start_time = time.time()

evaluate_model(model, custom_train_loader, criterion, optimizer, 150, scheduler, 1024 * 16, num_features)

elapsed_time = time.time() - start_time
print(f"Execution time: {elapsed_time:.6f} seconds")

In [ ]:
evaluate_model(model, custom_train_loader, criterion, optimizer, 150, scheduler, 1024 * 16, num_features)

In [ ]:
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler

optimizer = optim.Adam(model.parameters(), lr=0.001 * 10)
scheduler = lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.9995)

torch.cuda.synchronize()
start_time = time.time()

evaluate_model(model, custom_train_loader, criterion, optimizer, 150, scheduler, 1024 * 16, num_features)

elapsed_time = time.time() - start_time
print(f"Execution time: {elapsed_time:.6f} seconds")

In [ ]:
data = pd.read_csv('/kaggle/input/playground-series-s4e10/test.csv')
data['source'] = 0

print(data.columns)

X_test = data.drop(["id"], axis=1)
X_test['loan_percent_income_ratio'] = (X_test['person_income'] / X_test['loan_amnt'])
X_test['loan_to_income_ratio'] = X_test['loan_amnt'] / X_test['person_income']
X_test['financial_burden'] = X_test['loan_amnt'] * X_test['loan_int_rate']
X_test['loan_int_emp_interaction'] = X_test['loan_int_rate'] * X_test['person_emp_length']
X_test['debt_to_credit_ratio'] = X_test['loan_amnt'] / X_test['cb_person_cred_hist_length']

exclude_columns = [
    "loan_percent_income_ratio",
    "loan_to_income_ratio",
    "financial_burden",
    "debt_to_credit_ratio"
]

X_test = pd.get_dummies(X_test, drop_first=True)
for col in X_test.columns:
    if col not in exclude_columns:
        if (X_test[col] > 0).all():
            X_test[col] = np.log(X_test[col])

print(X_test.shape)
print(X_test.columns)

In [ ]:
X_scaled_test = x_scaler.transform(X_test)
print(X_scaled_test)

In [ ]:
print(X_scaled_test.shape)

In [ ]:
X_scaled_test_tensor = torch.tensor(X_scaled_test).float().to(device)
outputs = model(X_scaled_test_tensor)
print(outputs)

In [ ]:
probabilities = F.softmax(outputs, dim=1)
print(probabilities)

In [ ]:
positive_class_probs = probabilities[:, 1]
print(positive_class_probs)

In [ ]:
import pandas as pd

test_df = pd.read_csv('/kaggle/input/playground-series-s4e10/test.csv')
ids = test_df['id']

positive_class_probs = positive_class_probs.cpu().detach().numpy()

submission_df = pd.DataFrame({
    'id': ids,
    'loan_status': positive_class_probs
})

submission_df.to_csv('submission.csv', index=False)
print("Submission file created successfully.")

In [ ]:
print(custom_train_loader.train_labels_tensor)
print(model(custom_train_loader.train_data_tensor))

In [ ]:
import pandas as pd

# Path to the directory containing the model CSV files
file_path = '/kaggle/input/predictions/'

# List to store each model's DataFrame
dfs = []

# Loop to read each model file from model_0 to model_9
for i in range(10):
    file_name = f'{file_path}model_{i}.csv'
    df = pd.read_csv(file_name)
    dfs.append(df)

# Concatenate all DataFrames vertically
all_data = pd.concat(dfs)

# Calculate the average 'loan_status' for each 'id'
result_df = all_data.groupby('id', as_index=False)['loan_status'].mean()

# Save the result to a new CSV in the /kaggle/working directory for output
result_df.to_csv('submission.csv', index=False)

print("Averaging completed and saved to '/kaggle/working/average_loan_status.csv'")
print(result_df)

In [ ]:
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler

optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.9995)

torch.cuda.synchronize()
start_time = time.time()

evaluate_model(model, custom_train_loader, criterion, optimizer, 100, scheduler, 1024 * 16, num_features)

elapsed_time = time.time() - start_time
print(f"Execution time: {elapsed_time:.6f} seconds")

In [ ]:
evaluate_model(model, custom_train_loader, criterion, optimizer, 10, scheduler, 1024 * 16, num_features)